<div class="markdown-google-sans">

# **Emotion Detection Notebook**
</div>


>This notebook performs EDA and then preprocesses a Twitter dataset to train a baseline Naive Bayes Classifier, a bidirectional Bi-LSTM model, and fine-tuning BERT.

> The Bi-LSTM model is then integrated with streamlit and deployed as a web-app.

Outline:

I. [Setting up the environment](#eda)<br>
II. [Baseline Model : Naive Bayes](#naiveBayes)<br>
III. [Emotion Analysis using Bi-LSTM](#lstm)<br>
IV. [Emotion Analysis using BERT](#bert)<br>

<div class="markdown-google-sans">

# **Setting up the environment**
</div>
<a name="eda"></a>


## **Installing and importing dependencies**


In [2]:

!pip install huggingface
!pip install datasets
!pip install transformers
!pip install peft
!pip install evaluate
!pip install torch


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.0 MB/s eta 0:00:00


In [3]:

import numpy as np
import pandas as pd
import re, sys, os, csv, keras, pickle
from keras import regularizers, initializers, optimizers, callbacks
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten, Concatenate
from keras.layers import Conv1D, MaxPooling1D, Embedding, Dropout, LSTM, GRU, Bidirectional
from keras.models import Model
from keras import backend as K
from tensorflow.keras.layers import Layer, InputSpec
from tensorflow.keras.callbacks import ModelCheckpoint

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd '/content/gdrive/My Drive'


Mounted at /content/gdrive
/content/gdrive/My Drive


## Setting up Parameters

In [5]:
MAX_NB_WORDS = 56000 # max no. of words for tokenizer
MAX_SEQUENCE_LENGTH = 30 # max length of text (words) including padding
VALIDATION_SPLIT = 0.2
EMBEDDING_DIM = 200 # embedding dimensions for word vectors (word2vec/GloVe)
GLOVE_DIR = "glove.twitter.27B.200d.txt"


<div class="markdown-google-sans">

# **Baseline Model: Naive Bayes**
</div>
<a name="naiveBayes"></a>

In [6]:
import csv
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics

# Load the dataset
texts, labels = [], []

with open('emotion_data.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    for row in readCSV:
        texts.append(row[0])
        labels.append(row[1])

# Preprocess labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# Split the data into training and testing sets
texts_train, texts_test, labels_train, labels_test = train_test_split(texts, encoded_labels, test_size=0.2, random_state=42)

# Vectorize the text data using TF-IDF
vectorizer = TfidfVectorizer(max_features=5000)
X_train = vectorizer.fit_transform(texts_train)
X_test = vectorizer.transform(texts_test)

# Train a Multinomial Naive Bayes classifier
naive_bayes_classifier = MultinomialNB()
naive_bayes_classifier.fit(X_train, labels_train)

# Make predictions on the testing set
predictions = naive_bayes_classifier.predict(X_test)

# Evaluate the model
accuracy = metrics.accuracy_score(labels_test, predictions)
precision = metrics.precision_score(labels_test, predictions, average='weighted')
recall = metrics.recall_score(labels_test, predictions, average='weighted')
f1_score = metrics.f1_score(labels_test, predictions, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1_score)


Accuracy: 0.5405647691618108
Precision: 0.5648960028433084
Recall: 0.5405647691618108
F1 Score: 0.5168579956356953


Classes

In [7]:
classes = ["Neutral", "Happy", "Sad", "Love", "Anger"]

Pedictions

In [8]:
import pandas as pd

# Assuming 'texts_test' is the original text and 'predictions' is the predicted emotion labels
# You may need to adjust these variable names based on your actual variable names

# Map numeric predictions to emotion classes
predicted_emotions = [classes[prediction] for prediction in predictions]

# Create a DataFrame
df_result = pd.DataFrame({'Text': texts_test, 'Predicted Emotion': predicted_emotions})

# Display the DataFrame
df_result

,Text,Predicted Emotion
0,i m sunburnt on my arms and i have burnt my mo...,Sad
1,i m sure all you people with hangovers will be...,Happy
2,going to glasgow,Sad
3,watching the cavs,Happy
4,just hit the 3000 word mark for his geography ...,Sad
...,...,...
11150,omgosh i just saw hattie wake up must be summe...,Sad
11151,danget having problems with wlw and blog engin...,Sad
11152,getting ready to go to ikea all alone cuz no o...,Sad
11153,screenshots can t give you any other kind of p...,Happy


<div class="markdown-google-sans">

# **LSTM based model**
</div>
<a name="eda"></a>

<div class="markdown-google-sans">

## **Data processing**
</div>
<a name="eda"></a>

Importing the dataset

In [9]:
texts, labels = [], []
print("Reading from csv file...", end="")
with open('emotion_data.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    for row in readCSV:
        texts.append(row[0])
        labels.append(row[1])
print("\nDone!")

Reading from csv file...
Done!


Loading the tokenizer

In [10]:
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [11]:
tokenizer.analyzer=None
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
data_int = pad_sequences(sequences, padding='pre', maxlen=(MAX_SEQUENCE_LENGTH-5))
data = pad_sequences(data_int, padding='post', maxlen=(MAX_SEQUENCE_LENGTH))

Found 34359 unique tokens.


In [12]:
labels = to_categorical(np.asarray(labels)) # convert to one-hot encoding vectors
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Shape of data tensor: (55774, 30)
Shape of label tensor: (55774, 5)


In [13]:
df=pd.read_csv('emotion_data.csv')
df

,i m looking forward to going home tomorrow but i really wish it was for a different reason,2
0,just got to kansas city and excited for a fun ...,3
1,hey adt guess what my princelple s number plat...,1
2,b gt not the best song for her,0
3,the wind tried to hate on us today lol hello f...,4
4,i need to change my ways instead of just being...,3
...,...,...
55768,vancouver so classy canuck s,1
55769,gonna get my ticket to las vegas today gotta c...,1
55770,when life as you know it doesn t exist anymore,2
55771,my camera is brokennn,2


Training and validation split

In [14]:
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

In [15]:
x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

print('Number of entries in each category:-')
print("Training:\n",y_train.sum(axis=0))
print("Validation:\n",y_val.sum(axis=0))

Number of entries in each category:-
Training:
 [ 7700. 12446. 12981.  6555.  4938.]
Validation:
 [1943. 2998. 3270. 1693. 1250.]


Building the embedding matrix

In [16]:
embeddings_index = {}
f = open("glove.twitter.27B.200d.txt", encoding="utf8")
print("Loading GloVe...")
for line in f:
    values = line.split()
    word = values[0]
    embeddings_index[word] = np.asarray(values[1:], dtype='float32')
f.close()
print("Done.\nProceeding with Embedding Matrix...", end="")
embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
print("\nCompleted!")

Loading GloVe...
Done.
Proceeding with Embedding Matrix...
Completed!


In [17]:
print("Finished running setup.")


Finished running setup.


<div class="markdown-google-sans">

## **Building the model**
</div>
<a name="eda"></a>

Needed functions

In [18]:

def get_lr_metric(optimizer):
    def lr(y_true, y_pred):
        return optimizer.lr
    return lr
def initial_boost(epoch):
    if epoch==0: return float(8.0)
    elif epoch==1: return float(4.0)
    elif epoch==2: return float(2.0)
    elif epoch==3: return float(1.5)
    else: return float(1.0)
def step_cyclic(epoch):
    try:
        l_r, decay = 1.0, 0.0001
        if epoch%33==0:multiplier = 10
        else:multiplier = 1
        rate = float(multiplier * l_r * 1/(1 + decay * epoch))
        #print("Epoch",epoch+1,"- learning_rate",rate)
        return rate
    except Exception as e:
        print("Error in lr_schedule:",str(e))
        return float(1.0)

DL Model

In [19]:
# second embedding matrix for non-static channel
embedding_matrix_ns = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix_ns[i] = embedding_vector
print("Completed!")

Completed!


In [20]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')

# static channel
embedding_layer_frozen = Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix], input_length=MAX_SEQUENCE_LENGTH, trainable=False)
embedded_sequences_frozen = embedding_layer_frozen(sequence_input)

# non-static channel
embedding_layer_train = Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix_ns], input_length=MAX_SEQUENCE_LENGTH, trainable=True)
embedded_sequences_train = embedding_layer_train(sequence_input)

<div class="markdown-google-sans">

## **Model architecture**
</div>
<a name="eda"></a>

In [21]:
l_lstm1f = Bidirectional(LSTM(6,return_sequences=True,dropout=0.3, recurrent_dropout=0.0))(embedded_sequences_frozen)
l_lstm1t = Bidirectional(LSTM(6,return_sequences=True,dropout=0.3, recurrent_dropout=0.0))(embedded_sequences_train)
l_lstm1 = Concatenate(axis=1)([l_lstm1f, l_lstm1t])

In [22]:
l_conv_2 = Conv1D(filters=24,kernel_size=2,activation='relu')(l_lstm1)
l_conv_2 = Dropout(0.3)(l_conv_2)
l_conv_3 = Conv1D(filters=24,kernel_size=3,activation='relu')(l_lstm1)
l_conv_3 = Dropout(0.3)(l_conv_3)

l_conv_5 = Conv1D(filters=24,kernel_size=5,activation='relu',)(l_lstm1)
l_conv_5 = Dropout(0.3)(l_conv_5)
l_conv_6 = Conv1D(filters=24,kernel_size=6,activation='relu',kernel_regularizer=regularizers.l2(0.0001))(l_lstm1)
l_conv_6 = Dropout(0.3)(l_conv_6)

l_conv_8 = Conv1D(filters=24,kernel_size=8,activation='relu',kernel_regularizer=regularizers.l2(0.0001))(l_lstm1)
l_conv_8 = Dropout(0.3)(l_conv_8)

conv_1 = [l_conv_6,l_conv_5, l_conv_8,l_conv_2,l_conv_3]

l_lstm_c = Concatenate(axis=1)(conv_1)

In [23]:
l_conv_4f = Conv1D(filters=12,kernel_size=4,activation='relu',kernel_regularizer=regularizers.l2(0.0001))(embedded_sequences_frozen)
l_conv_4f = Dropout(0.3)(l_conv_4f)
l_conv_4t = Conv1D(filters=12,kernel_size=4,activation='relu',kernel_regularizer=regularizers.l2(0.0001))(embedded_sequences_train)
l_conv_4t = Dropout(0.3)(l_conv_4t)

l_conv_3f = Conv1D(filters=12,kernel_size=3,activation='relu',)(embedded_sequences_frozen)
l_conv_3f = Dropout(0.3)(l_conv_3f)
l_conv_3t = Conv1D(filters=12,kernel_size=3,activation='relu',)(embedded_sequences_train)
l_conv_3t = Dropout(0.3)(l_conv_3t)

l_conv_2f = Conv1D(filters=12,kernel_size=2,activation='relu')(embedded_sequences_frozen)
l_conv_2f = Dropout(0.3)(l_conv_2f)
l_conv_2t = Conv1D(filters=12,kernel_size=2,activation='relu')(embedded_sequences_train)
l_conv_2t = Dropout(0.3)(l_conv_2t)

conv_2 = [l_conv_4f, l_conv_4t,l_conv_3f, l_conv_3t, l_conv_2f, l_conv_2t]

l_merge_2 = Concatenate(axis=1)(conv_2)
l_c_lstm = Bidirectional(LSTM(12,return_sequences=True,dropout=0.3, recurrent_dropout=0.0))(l_merge_2)

In [24]:
l_merge = Concatenate(axis=1)([l_lstm_c, l_c_lstm])
l_pool = MaxPooling1D(4)(l_merge)
l_drop = Dropout(0.5)(l_pool)
l_flat = Flatten()(l_drop)
l_dense = Dense(26, activation='relu')(l_flat)
preds = Dense(5, activation='softmax')(l_dense)

In [25]:
model = Model(sequence_input, preds)
adadelta = optimizers.legacy.Adadelta(lr=0.9, rho=0.95, epsilon=None, decay=0.002)
lr_metric = get_lr_metric(adadelta)
model.compile(loss='categorical_crossentropy', optimizer=adadelta, metrics=['acc'])

/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/legacy/adadelta.py:82: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [26]:
tensorboard = callbacks.TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=16, write_grads=True , write_graph=True)
model_checkpoints = ModelCheckpoint("checkpoint-{val_acc:.4f}.h5", monitor='val_acc', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=0)
lr_schedule = callbacks.LearningRateScheduler(initial_boost)

<div class="markdown-google-sans">

## **Model saving and summary**
</div>
<a name="eda"></a>

Loading the model

In [27]:
model = keras.models.load_model("BalanceNet.h5")


Training the model

In [28]:
# print("Training Progress:")
# model_log = model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=10, batch_size=100, callbacks=[tensorboard, model_checkpoints])
# pd.DataFrame(model_log.history).to_csv("history-balance.csv")

Plotting the accuracy and the loss for train and validation

In [29]:
model.save('BalanceNet_trained.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [30]:
model = keras.models.load_model("BalanceNet_trained.h5")


In [31]:
import matplotlib.pyplot as plt

accuracy = model_log.history['acc']
val_accuracy = model_log.history['val_acc']
loss = model_log.history['loss']
val_loss = model_log.history['val_loss']

epochs = range(1, len(accuracy)+1)

plt.plot(epochs, accuracy, 'g', label='Training accuracy')
plt.plot(epochs, val_accuracy, 'r', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'g', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

NameError: name 'model_log' is not defined

In [32]:
from keras.models import load_model
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
%config InlineBackend.figure_format = 'retina'
import itertools, pickle

with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

classes = ["Neutral", "Happy", "Sad", "Love", "Anger"]

<div class="markdown-google-sans">

## **Predicting**
</div>
<a name="eda"></a>

In [ ]:
#Predicting emotions from paragraphs and sentences.

text = ["For instance, giving a kiss to your younger sibling daily after waking up in the morning and showing him how much you love them. For some happiness means loving life and seeing others happy. While some finds happiness in writing stories. Some conquer happiness in being simple yet the best person they can ever be. Everyone has their own unique way to feel happy by finding things that they never expected to find.", # happy
        "Love is the key to happiness. We all want to lead a happy life. People look around for happiness in power, fashion, wealth, drugs etc. But these things can only give temporary pleasures. The power of love can create miracles. Love can create unity among nations and its citizens. Love is the most beautiful feeling in the world. Love has given different meaning by different people depending upon how they have experienced this wonderful feeling.", # love
        "One day I was studying in my room when, all of a sudden, i heard hot words being exchanged between two persons in the street. I paid no attention, thinking it would be a minor quarrel but soon I heard the voices of a large number of people. I peeped from the window and saw that there was a street quarrel. I went downstairs and reached the spot in the twinkling of an eyes. I was at my wits end on seeing that both of them had come to blows. The people were standing around them and enjoying their quarrel but none tried to pacify them.", # sad
        "I am so angry at you!!!!!", # anger
        "you ve hit a new low with a danger of blm fascist slogan please stop it before too late stop", # anger
        "I love my doggg", # love
        "I think i'm gonna be sick :'‑(", # sad
        "I hate you so much", # anger
        "I'm at work", # neutral
        "@TheTombert i was watching Harpers Island, lol... there was no vodka involved", # neutral
        "sometimes i wish things could go back to the way they were the beginning of last summer", # sad
        "it's your 18th birthday finally!!! yippeeeee", # happy
        "still waiting in line", # neutral
        "aarrgghh - fu*k.....a hose has leaked water all over the new floating floor", # anger
        "that b*tch is so ugly", # anger
        "oh no he is hospitalised!!!", # sad
        "i'm very angry right now",
        "I can't be any happier"
               ]

In [33]:
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

data=pd.read_csv('emotion_data_test.csv')

predictions = []
for example in data['text']:
    if 'happy' in example.lower():
        predictions.append('happy')
    elif 'love' in example.lower():
        predictions.append('love')
    elif 'sad' in example.lower():
        predictions.append('sad')
    elif 'anger' in example.lower() or 'angry' in example.lower():
        predictions.append('anger')
    else:
        predictions.append('neutral')

# Add the predictions to the DataFrame
data['predicted_emotion'] = predictions

# Calculate precision
precision = precision_score(data['emotion'], data['predicted_emotion'], average='weighted')

# Calculate recall
recall = recall_score(data['emotion'], data['predicted_emotion'], average='weighted')

# Calculate F1 score
f1 = f1_score(data['emotion'], data['predicted_emotion'], average='weighted')

# Calculate accuracy
accuracy = accuracy_score(data['emotion'], data['predicted_emotion'])

# Display the metrics
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Accuracy:", accuracy)

Precision: 0.5694444444444444
Recall: 0.4444444444444444
F1 Score: 0.42962962962962964
Accuracy: 0.4444444444444444


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
tokenizer.analyzer=None
sequences_test = tokenizer.texts_to_sequences(text)
data_int_t = pad_sequences(sequences_test, padding='pre', maxlen=(MAX_SEQUENCE_LENGTH-5))
data_test = pad_sequences(data_int_t, padding='post', maxlen=(MAX_SEQUENCE_LENGTH))
y_prob = model.predict(data_test)
for n, prediction in enumerate(y_prob):
    pred = y_prob.argmax(axis=-1)[n]
    print(text[n],"\nPREDICTION:",classes[pred],"\n")

<div class="markdown-google-sans">

# **Fine tuning of bert**
</div>
<a name="eda"></a>

In [ ]:
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch.nn.functional import softmax
import torch

# Load your local CSV file
file_path = 'emotion_data.csv'
df = pd.read_csv(file_path)

# Check the structure of your CSV file
print(df.head())

# Tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Tokenize the text using the first column (adjust the index if needed)
tokenized = tokenizer(df.iloc[:, 0].astype(str).tolist(), padding=True, truncation=True, return_tensors='pt')

# Encode labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(df.iloc[:, 1].astype(str).tolist())

# Create a PyTorch Dataset
class EmotionDataset(Dataset):
    def __init__(self, input_ids, attention_mask, labels):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_mask[idx],
            'labels': self.labels[idx]
        }

# Create Dataset instances
train_dataset, valid_dataset = train_test_split(
    EmotionDataset(input_ids=tokenized['input_ids'],
                   attention_mask=tokenized['attention_mask'],
                   labels=encoded_labels),
    test_size=0.2,
    random_state=42
)

# Model
num_labels = len(label_encoder.classes_)
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_labels)

# Training parameters
batch_size = 8
num_epochs = 2
learning_rate = 5e-5

# DataLoaders
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size)

# Optimizer and Scheduler
optimizer = AdamW(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9)

# Training Loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(num_epochs):
    model.train()
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    model.eval()
    with torch.no_grad():
        for batch in valid_dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            # Evaluate as needed

    scheduler.step()

# Save the fine-tuned model
model.save_pretrained("fine_tuned_emotion_model")
tokenizer.save_pretrained("fine_tuned_emotion_model")
